In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from tqdm import tqdm
from datasets import GRUDFdataset
from grudf import GRU_DF, preprocess_dataset
from grudf_Loss import WeightedBCELoss
from accuracies import accuracies

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = np.load('data_X.npy')
y = np.load('data_y.npy')

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
train_index, test_index = next(sss.split(X, y))
val_index, test_index = next(StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0).split(X[test_index], y[test_index]))

X, delta, delta_future, M, last_observation, next_observation, empirical_mean = preprocess_dataset(X)

train_dataset = GRUDFdataset(X[train_index], delta[train_index], delta_future[train_index],
                             M[train_index], last_observation[train_index], next_observation[train_index], empirical_mean[train_index], y[train_index])
val_dataset = GRUDFdataset(X[val_index], delta[val_index], delta_future[val_index], 
                           M[val_index], last_observation[val_index], next_observation[val_index], empirical_mean[val_index], y[val_index])
test_dataset = GRUDFdataset(X[test_index], delta[test_index], delta_future[test_index], 
                            M[test_index], last_observation[test_index], next_observation[test_index], empirical_mean[test_index], y[test_index])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_size = 5
hidden_size = 16
output_size = 1
lr_rate = 0.001

model = GRU_DF(input_size=input_size, hidden_size=hidden_size)
model.to(device)

w_p = torch.tensor(y[train_index].sum() / len(y[train_index]), dtype=torch.float32)
criterion = WeightedBCELoss(w_p)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

early_stopping_patience = 20
early_stopping_counter = 0
best_val_loss = float('inf')
model_path = 'best_model_GRUDF.pth'

num_epochs = 1000
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        new_batch = []
        for item in batch:
            new_batch.append(item.to(device))
        input_batch = tuple(new_batch[:-1])
        y_batch = new_batch[-1]
        M_batch = new_batch[3]
        predictions = model(*input_batch).squeeze()
        loss = criterion(predictions, y_batch, M_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * input_batch[0].size(0)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            new_batch = []
            for item in batch:
                new_batch.append(item.to(device))
            input_batch = tuple(new_batch[:-1])
            y_batch = new_batch[-1]
            M_batch = new_batch[3]
            predictions = model(*input_batch).squeeze()
            loss = criterion(predictions, y_batch, M_batch)
            val_loss += loss.item() * input_batch[0].size(0)

    train_loss = running_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    scheduler.step()

    if epoch % 10 == 0:
        torch.save(model.state_dict(), f'best_model_GRUDF_{epoch}.pth')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), model_path)
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break

model.load_state_dict(torch.load(model_path))

model.eval()
test_losses = []
test_labels = []
test_predictions = []

with torch.no_grad():
    for batch in val_loader:
        new_batch = []
        for item in batch:
            new_batch.append(item.to(device))
        input_batch = tuple(new_batch[:-1])
        y_batch = new_batch[-1]
        M_batch = new_batch[3]
        outputs = model(*input_batch).squeeze()
        loss = criterion(outputs, y_batch, M_batch)
        test_losses.append(loss.item())
        test_predictions.extend(outputs.tolist())
        test_labels.extend(y_batch.tolist())

average_test_loss = sum(test_losses) / len(test_losses)
print(f'Average test loss: {average_test_loss:.4f}')

test_predictions = np.array(test_predictions)
test_labels = np.array(test_labels)

accuracies(test_labels, test_predictions)

/home/ty/Gatech/Fa2023/Deep_Learning_Final_Project/grud.py:117: RuntimeWarning: Mean of empty slice
  empirical_mean = np.nanmean(X, axis=1)
/home/ty/Gatech/Fa2023/Deep_Learning_Final_Project/grudf.py:123: RuntimeWarning: Mean of empty slice
  empirical_mean_1 = np.nanmean(X, axis=1)
  0%|          | 1/1000 [01:03<17:31:47, 63.17s/it]

Epoch 1/1000, Training Loss: 0.0662, Validation Loss: 0.0605


  0%|          | 2/1000 [02:07<17:43:27, 63.94s/it]

Epoch 2/1000, Training Loss: 0.0576, Validation Loss: 0.0549


In [13]:
model.load_state_dict(torch.load(model_path))

model.eval()
test_losses = []
test_labels = []
test_predictions = []

Average test loss: 0.3622
AUC score: 0.8319
Accuracy: 0.7537
Sensitivity: 0.7987
Specificity: 0.7438
